In [1]:
# dans ce code on cherche à définir plusieurs zones à une échéance fixe
# v2: possibilité de selectionner +sieurs zones pour un temps sensible critique donné 
# import sklearn.metrics
import numpy as np
import glob
import xarray as xr 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib
import pandas as pd
import time
import glob
import sys, os
import string
from pathlib import Path # pour windows 
sys.path.insert(0, os.path.abspath('./lib'))

from lib import hss,precision #,far,f1, pod,pofd
from lib import create_combination_subzones, create_nc_mask_NSEO
from lib import find_neighbours,get_optimal_subzone_v2, group_masks_size, select_group_mask, get_WME_legend, get_not_included_masks

import stageemi
import stageemi.dev.distance_wwmf as distance_wwmf

In [2]:
matplotlib.rcParams['legend.handlelength'] = 0
matplotlib.rcParams['legend.numpoints'] = 1

In [37]:
''' input '''
# date = '2020012600'
date = '2020030600'
list_name = ['compas','agat','compas_asym','agat_asym'] # pour agreger le temps sensible

mask_sympo = True
mask_geographique = False
dir_fig = '../figures/total/'
nsubzonesMax = 7
plot_results = True
if date == '2020012600':
    echeance_dict = {
        '38':[44,12,3,46,43,25,30],
        '29':[32,39,20,33,13],  
        '34':[1,5,6,4 ,10, 20,30], 
        '41':[45,4,44,5,20,30]
    }
if date == '2020030600':
    echeance_dict = {
#         '38':[29,3,1,4,36],
#         '41':[18],
        '29':[1,5,3],  
        '34':[31,6,16,29,30], 
    }
    
for dep_id in echeance_dict.keys():
    echeance_list = echeance_dict[dep_id]
    print('dep_id',dep_id)
    ''' lecture du mask '''
    if mask_sympo and not mask_geographique: 
        t1 = time.time()
        fname_out = '../GeoData/zones_sympo_multiples/'+dep_id+'_mask_zones_sympos.nc'
        if not os.path.exists(fname_out): 
            dir_mask = '/home/mrpa/borderiesm/stageEMI/Codes/StageEMI/Masques_netcdf/ZONE_SYMPO/'
            list_subzones = glob.glob(dir_mask + dep_id +'*.nc')
            n_subzones = len(list_subzones)  # nombre de zones sympos initiales
            lst_subzones = [zone[-7:-3] for zone in list_subzones]
            ds_mask = create_combination_subzones(dir_mask,dep_id,lst_subzones,fname_out,degre5=True) 
        else: 
            ds_mask = xr.open_dataset(fname_out,chunks={"id":1})
            ds_mask["latitude"]  = ds_mask["latitude"].round(5)
            ds_mask["longitude"] = ds_mask["longitude"].round(5)
        print(time.time() - t1)

    if mask_geographique and not mask_sympo: 
        dir_mask  = '/home/mrpa/borderiesm/stageEMI/Codes/stageemi/stageemi/GeoData/nc_departement/'
        if   dep_id == '38': dep = 'FRK24'
        elif dep_id == '41': dep = 'FRB05'
        elif dep_id == "34": dep = 'FRJ13'
        elif dep_id == '29': dep = "FRH02"
        else: 
            print('remplir la bonne valeur pour le dep')
            sys.exit()
        dep_file  = dir_mask + dep +'.nc' 
        fname_out = '/home/mrpa/borderiesm/stageEMI/Codes/StageEMI/Masques_netcdf/ZONE_SYMPO_MULTIPLE/'+ dep_id+'_'+dep+'_mask_NSEO.nc'

        if not os.path.exists(fname_out):
            ds_mask = create_nc_mask_NSEO(dep_file,fname_out)
        else:
            ds_mask = xr.open_dataset(fname_out,chunks={"id":1})
            ds_mask["latitude"]  = ds_mask["latitude"].round(5)
            ds_mask["longitude"] = ds_mask["longitude"].round(5)
    ''' lecture arome '''
    fname = "../WWMF/" + date+'0000__PG0PAROME__'+'WWMF'+'__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc'

    ds = xr.open_dataset(fname,chunks={"step":1}).isel(step = echeance_list)
    ds['latitude']  = ds['latitude'].round(5)
    ds['longitude'] = ds['longitude'].round(5)
    
    if mask_sympo and not mask_geographique: 
        ds_dep_tot = (ds*ds_mask.mask.sel(id="departement").drop("id"))
    if mask_geographique and not mask_sympo: 
        ds_dep_tot = (ds*ds_mask.mask.sel(id="mask").drop("id"))
    if date == '2020030600':
        ds_dep_tot = ds_dep_tot.rename({'paramId_0':'unknown'})
        
    ''' calcul des temps agrégés '''
    ds_distance_dict = {}
    for name in list_name:
        ds_distance         = distance_wwmf.get_pixel_distance_dept(ds_dep_tot,name)
        ds_distance_chunk   = ds_distance.chunk({"step":1}) 
        ds_distance_dict[name] = (ds_distance_chunk * ds_mask.mask).sum(['latitude',"longitude"]).compute()
    print('fin calcul distance')
    
    var_name = 'wme_arr'
    for icheance,echeance in enumerate(echeance_list): 
        print(echeance)
        fname_out = '../zonageWME/'+dep_id+'_'+date+'_'+str(echeance)+'.csv'
        if os.path.exists(fname_out):
            print(fname_out,'existe')
            continue
        
        tdeb = time.time()
        ''' on restreint la liste des WME pour le zonage '''
        ds_dep = ds_dep_tot.isel(step = icheance).copy()
        # on regroupe 'Très nuageux/Couvert' et 'Nuageux'
        ds_dep = ds_dep.where(~((ds_dep[var_name].values == 2) + (ds_dep[var_name].values == 3) ), 2)

        # on regroupe ensemble neige (10) et neige faible (7)
        ds_dep = ds_dep.where(~((ds_dep[var_name].values == 7) + (ds_dep[var_name].values == 10)), 10)
        
        # on regroupe ensemble pluie (8) et pluie faible (6)
        ds_dep = ds_dep.where(~((ds_dep[var_name].values == 8) + (ds_dep[var_name].values == 6)),8)

        # on regroupe ensemble qlqs averses (12) et averses (14), et qlqs averses de neige (13)
        ds_dep = ds_dep.where(~((ds_dep[var_name].values == 12) + (ds_dep[var_name].values == 13)
                                  + (ds_dep[var_name].values == 14 )),14)

        # on regroupe ensemble averses Orageuses (16) et Orages  (18)
        ds_dep = ds_dep.where(~((ds_dep[var_name].values == 16) + (ds_dep[var_name].values == 18)),18)

        file_CodesWWMF = '../utils/CodesWWMF.csv'
        cible_list,legend_list = get_WME_legend(file_CodesWWMF, ds_dep)
        print(cible_list,legend_list)

        ''' zonage '''
        listCible    = cible_list[::-1]
        legend_cible = [] # pour stocker la légende du code WME
        listMasksNew = ds_mask.id.values # on commence avec l'ensemble des masks

        # liste de zones sympos initiales (pour checker à la fin si on a une info sur toutes les zones du département)
        list_zones_sympos_initiales = [zone for zone in ds_mask.id.values if len(zone) == 4]

        nsubzones    = 0
        zones_cibles = {}
        score_zones_cibles = {}
        if len(listCible) == 0 : # si un département a le même temps sensible partout
            zones_cibles[listCible[0]] = 'departement'
        else: 
            for icible,cible in enumerate(listCible):
                if nsubzones > nsubzonesMax: 
                    print('nombre de sous-zones trop grand')
                    break 
                if nsubzones >1: 
                    # pour éviter que departement ne soit selectionné alors que des sous-zones de departement aient déjà été selectionnées.
                    listMasksNew = [element for element in listMasksNew if element !='departement']

                if len(listMasksNew)>60:
                    #  on regroupe les masks selon leur taille pour aller plus vite 
                    groupe1,groupe2,groupe3,taille1,taille2  = group_masks_size(listMasksNew,ds_mask)
                    # on selectionne le groupement de zones qui match l'objet météo
                    groupe_mask_select = select_group_mask(ds_dep,cible,groupe1,groupe2,groupe3,taille1,taille2)
                else: 
                    # on considère l'ensemble des masks
                    groupe_mask_select = ds_mask.mask.sel(id=listMasksNew)
                # on selectionne la zone optimale (selon le hss et la précision)
                zones_optimales,score_hss,score_precision=get_optimal_subzone_v2(ds_dep, groupe_mask_select,cible,ds_mask)
                if len(zones_optimales)==0:
                    # pas de zone sélectionnée pour ce temps sensible
                    continue
                else: 
                    legend_cible.append(legend_list[::-1][icible])
                    score_zones_cibles[cible] = score_hss
                    zones_cibles[cible] = zones_optimales 
                    nsubzones +=1                          

                ''' on check que la somme des zones n'est pas déjà égale au departement '''
                if  (nsubzones== 1) and (len(zones_cibles[cible]) == 1) :
                    ds_temp  = ds_mask.sel(id=zones_cibles[cible][0]).mask.copy()

                elif (nsubzones== 1) and (len(zones_cibles[cible]) > 1): 
                    ds_temp  = ds_mask.sel(id=zones_cibles[cible][0]).mask.copy() 
                    ds_temp.values[(ds_temp.values == 1) + (ds_mask.sel(id=zones_cibles[cible][1]).mask.values ==1) ] = 1
                else: 
                    for zone in zones_cibles[cible]:
                        print(zone)
                        ds_temp.values[(ds_temp.values == 1) + (ds_mask.sel(id=zone).mask.values ==1) ] = 1

                somme = np.sum((ds_temp.values == 1)&( ds_mask.sel(id='departement').mask.values== 1))
                tailleDep = np.sum( ds_mask.sel(id='departement').mask.values== 1)
                if somme == tailleDep: 
                    print('on a atteint la taille du departement')
                    break
                # on récupère les zones non-incluses dans la zone sélectionnée
                for zone in zones_cibles[cible]:
                    listMasksNew, lst_mask_included = get_not_included_masks(ds_mask.mask.sel(id=zone)
                                                    ,listMasksNew,ds_mask,flag_strictly_included=False)
            # fin boucle sur cible
            ''' on vérifie que toutes les zones du département sont dans les zones selectionnées '''
            list_zones_select = sum([zones_cibles[cible] for cible in zones_cibles.keys()],[]) # [zones_cibles[cible] for cible in zones_cibles.keys()]
            zones_restantes = []
            for zone_sympo in list_zones_sympos_initiales:
                n = 0
                for zone_select in list_zones_select: 
                    if zone_sympo in zone_select:
                        n+=1
                if n == 0 : 
                    zones_restantes.append(zone_sympo)
        
        print(zones_cibles)          
        '''save results in csv'''
        print('saving results')
        
        d = { 'zone':sum([zones_cibles[cible] for cible in zones_cibles.keys()],[]), 
            'cible_wme':sum([[cible]  if len(zones_cibles[cible])==1 else [cible,cible] for cible in zones_cibles.keys()],[]),
            'hss' : sum([score_zones_cibles[cible] for cible in zones_cibles.keys()],[])}

        if len(zones_restantes)>0:
            d['zone'] += zones_restantes
            d['hss'] += [np.nan for i in range(len(zones_restantes))]
            d['cible_wme'] += [np.nan for i in range(len(zones_restantes))]
        for name in list_name:
            d[name] =  ds_distance_dict[name].wwmf_2[ds_distance_dict[name].argmin("wwmf_2")].sel(id=d['zone']).isel(step=icheance).values
        pd.DataFrame(data=d).to_csv(fname_out)
        
        ''' plot '''
        if not plot_results: 
            continue
        print('plot')
        X,Y = np.meshgrid( ds_mask.longitude.values,ds_mask.latitude.values)
        listMasks = [ds_mask.sel(id=id_ref) for id_ref in list_zones_sympos_initiales]

        legende = string.ascii_lowercase
        patches = []
        fig,axes = plt.subplots(nrows=1,ncols =3,figsize  = (15,5))
        ax = axes.flat

        fig.subplots_adjust(wspace=0.3)
        var2plot_lst = ['unknown','wme_arr','w1_arr']
        varmin_lst   = [0,1,0]
        varmax_lst   = [99,19,30]
        for iplot in range(3):
            var2plot = ds_dep_tot[var2plot_lst[iplot]].isel(step = icheance) 
            if iplot == 0 : 
                cmap  = matplotlib.cm.jet
            else: 
                cmap = matplotlib.cm.tab20b
                     
            varmin   = varmin_lst[iplot]
            varmax   = varmax_lst[iplot] + 1        
            clevs    = np.arange(varmin,varmax+1,1)
            cs       = var2plot.plot.imshow(ax = ax[iplot],cmap=cmap,levels=clevs)
            for icible,cible in enumerate(zones_cibles):
                for zone_select in  zones_cibles[cible] :
                    mask_ref = ds_mask.sel(id = zone_select)

                    list_neighbours = find_neighbours(mask_ref,listMasks)
                    lst_mask_not_included, lst_mask_included = get_not_included_masks(mask_ref.mask, list_neighbours,ds_mask,flag_strictly_included=True)
                    for neighbours in lst_mask_not_included:
                        ind = np.where((mask_ref.mask.values == 1) & (ds_mask.sel(id=neighbours).mask.values == 1))
                        ax[iplot].scatter(X[ind],Y[ind],color='k',s=6)
                    # 
                    # ajout de la legende
                    indice_mask_ref = np.where(mask_ref.mask.values == 1)

                    ax[iplot].text(X[indice_mask_ref].mean(),Y[indice_mask_ref].mean(),s=legende[icible],color='k',fontsize=15)
                    ax[iplot].set_title(date+' + {} h'.format(echeance))
                    if iplot ==0:
                        label = zone_select +': '+ legend_cible[icible] + ' ({})'.format(cible)
                        # ajout de l'agregation: 
                        for name in list_name: 
                            val_agrege = ds_distance_dict[name].wwmf_2[ds_distance_dict[name].argmin("wwmf_2")].sel(id=zone_select).isel(step=icheance).values
                            label += ' {}:{}'.format(name,val_agrege)
                if iplot == 0:
                    patches.append(mlines.Line2D([],[],label = label,marker='${}$'.format(legende[icible]),color='black'))
        lgd = ax[2].legend(handles=patches,bbox_to_anchor=(0.5,-0.2), loc='upper right',labelspacing =2,fontsize = 14)
        fig.tight_layout()
        fname_fig = dir_fig + 'v5_zonage_'+dep_id+date+'_'+str(echeance)+'.png'
        print(fname_fig)
        fig.savefig(fname_fig,dpi=400,bbox_inches='tight',format='png',bbox_extra_artists=(lgd,),)
        plt.clf()
        plt.close('all')
        print('temps',time.time()-tdeb)
    print()



dep_id 29
0.05296826362609863
compas 27.310178518295288
agat 46.60824251174927
compas_asym 25.7461040019989
agat_asym 36.59981298446655
fin calcul distance
1
../zonageWME/29_2020030600_1.csv existe
5
[ 1.  2.  8. 14.] ['Clair/Peu nuageux', 'Nuageux', 'Pluie', 'Averses']
objet dans groupe 1
zone optimales ['2902+2904'] [0.3393709996743765] [0.36750164798945284]
objet dans groupe 1
zone optimales ['2905+2906+2907+2908'] [0.3011470786949041] [0.481039755351682]
2905+2906+2907+2908
zone optimales ['2901+2903+2981+2982'] [0.4602756357030696] [0.751294305057746]
2901+2903+2981+2982
zone optimales ['2983'] [0.48501163609852904] [0.6690058479532164]
2983
{14.0: ['2902+2904'], 8.0: ['2905+2906+2907+2908'], 2.0: ['2901+2903+2981+2982'], 1.0: ['2983']}
saving results
plot


<ipython-input-37-d221fd5cc286>:268: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  fig.tight_layout()


../figures/total/v5_zonage_292020030600_5.png
temps 808.1119112968445
3
[ 2.  4.  8. 14.] ['Nuageux', 'Brume/Brouillard/Grisaille', 'Pluie', 'Averses']
objet dans groupe 1
zone optimales ['2901+2903+2981'] [0.19509402039868934] [0.2799405646359584]
objet dans groupe 3
zone optimales ['2902+2904+2905+2906+2907+2908+2983'] [0.21277972193050462] [0.8876960022602063]
2902+2904+2905+2906+2907+2908+2983
pas de zones homogène pour 4.0
zone optimales [] [] []
zone optimales ['2982'] [0.17797393273802836] [0.13703284258210646]
2982
{14.0: ['2901+2903+2981'], 8.0: ['2902+2904+2905+2906+2907+2908+2983'], 2.0: ['2982']}
saving results
plot


<ipython-input-37-d221fd5cc286>:268: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  fig.tight_layout()


../figures/total/v5_zonage_292020030600_3.png
temps 637.0357148647308

dep_id 34
0.2198634147644043
compas 83.294362783432
agat 79.02120852470398
compas_asym 49.998127937316895
agat_asym 65.76839208602905
fin calcul distance
31
[ 1.  2.  4.  8. 10. 14.] ['Clair/Peu nuageux', 'Nuageux', 'Brume/Brouillard/Grisaille', 'Pluie', 'Neige', 'Averses']
objet dans groupe 1
zone optimales ['3401'] [0.30309672941669613] [0.20089285714285715]
objet dans groupe 1
zone optimales ['3403'] [0.026094851143391343] [0.02366345311130587]
3403
objet dans groupe 1
zone optimales ['3402'] [0.16541384009507143] [0.19675456389452334]
3402
objet dans groupe 1
pas de zones homogène pour 4.0
zone optimales [] [] []
objet dans groupe 1
pas de zones homogène pour 2.0
zone optimales [] [] []
objet dans groupe 3
zone optimales ['3404+3405+3406+3407+3408+3483+3481+3482'] [0.6578776487366182] [0.9854865085854456]
3404+3405+3406+3407+3408+3483+3481+3482
{14.0: ['3401'], 10.0: ['3403'], 8.0: ['3402'], 1.0: ['3404+3405+340

...

../figures/total/v5_zonage_342020030600_16.png
temps 1515.540400981903
29
[ 1.  2.  4.  8. 10. 14.] ['Clair/Peu nuageux', 'Nuageux', 'Brume/Brouillard/Grisaille', 'Pluie', 'Neige', 'Averses']
objet dans groupe 1
zone optimales ['3401'] [0.19119778626960074] [0.11830357142857142]
objet dans groupe 1
pas de zones homogène pour 10.0
zone optimales [] [] []
objet dans groupe 1
zone optimales ['3403'] [0.1627109515471446] [0.17791411042944785]
3403
objet dans groupe 1
pas de zones homogène pour 4.0
zone optimales [] [] []
objet dans groupe 1
zone optimales ['3402'] [0.43662845438017545] [0.48073022312373226]
3402
objet dans groupe 3
zone optimales ['3404+3405+3406+3407+3408+3483+3481+3482'] [0.6727962448249429] [0.9977514309076042]
3404+3405+3406+3407+3408+3483+3481+3482
{14.0: ['3401'], 8.0: ['3403'], 2.0: ['3402'], 1.0: ['3404+3405+3406+3407+3408+3483+3481+3482']}
saving results
plot


<ipython-input-37-d221fd5cc286>:268: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  fig.tight_layout()


../figures/total/v5_zonage_342020030600_29.png
temps 1919.2777075767517
30
[ 1.  2.  4.  8. 10. 14.] ['Clair/Peu nuageux', 'Nuageux', 'Brume/Brouillard/Grisaille', 'Pluie', 'Neige', 'Averses']
objet dans groupe 1
zone optimales ['3401'] [0.1111090695956697] [0.07366071428571429]
objet dans groupe 1
pas de zones homogène pour 10.0
zone optimales [] [] []
objet dans groupe 1
zone optimales ['3402'] [0.07928506948761331] [0.11156186612576065]
3402
objet dans groupe 1
pas de zones homogène pour 4.0
zone optimales [] [] []
objet dans groupe 1
zone optimales ['3403'] [0.20395203581387672] [0.2953549517966696]
3403
objet dans groupe 3
zone optimales ['3404+3405+3406+3407+3408+3483+3481+3482'] [0.6378739343033336] [0.9785363859362224]
3404+3405+3406+3407+3408+3483+3481+3482
{14.0: ['3401'], 8.0: ['3402'], 2.0: ['3403'], 1.0: ['3404+3405+3406+3407+3408+3483+3481+3482']}
saving results
plot


<ipython-input-37-d221fd5cc286>:268: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  fig.tight_layout()


../figures/total/v5_zonage_342020030600_30.png
temps 1954.7807004451752



In [31]:
print(zones_cibles)
print(score_zones_cibles)
print(zones_restantes)
[np.nan for i in range(len(zones_restantes)+1)]

{14.0: ['2902+2903+2907+2982'], 8.0: ['2901+2981']}
{14.0: [0.27913830535920314], 8.0: [0.13401926175441523]}
['2904', '2905', '2906', '2908', '2983']


[nan, nan, nan, nan, nan, nan]